In [39]:
import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

## Каналы-комментаторы

In [2]:
%%sql
select
    channels.channel_id,
    channels.channel_name,
    channels.subscribers,
    case
        when subscribers < 100 then 'lover_small'
        when subscribers < 500 then 'upper_small'
        when subscribers < 2000 then 'lower_medium'
        when subscribers < 10000 then 'upper_medium'
        else 'large'
    end as channel_segment,

    comments.post_id,
    comments.comment_date,
    comments.author_uuid,
    comments.author_username,
    comments.author_title,
    case
        when comments.author_username = 'user' then 'user'
        else 'channel'
    end as author_type,
    case
        when channels.channel_name = comments.author_username then true
        else false
    end as is_internal,

    posts.post_date,
    strftime('%s', comments.comment_date) - strftime('%s', posts.post_date) as time_to_comment_sec

from comments
left join channels on comments.channel_id=channels.channel_id
left join posts on comments.channel_id=posts.channel_id and comments.post_id=posts.post_id


,channel_id,channel_name,subscribers,channel_segment,post_id,comment_date,author_uuid,author_username,author_title,author_type,is_internal,post_date,time_to_comment_sec
0,1542820616,tagir_analyzes,10564,large,341,2025-09-05 07:00:33.000000,490128b8-36c5-5539-94fa-6edd4d20fc16,user,user,user,0,2025-09-03 17:15:05.000000,135928
1,1542820616,tagir_analyzes,10564,large,341,2025-09-04 19:14:39.000000,bdb4cafa-9e2a-5a04-ae3e-2c4b284a45d5,user,user,user,0,2025-09-03 17:15:05.000000,93574
2,1542820616,tagir_analyzes,10564,large,341,2025-09-04 19:13:08.000000,bdb4cafa-9e2a-5a04-ae3e-2c4b284a45d5,user,user,user,0,2025-09-03 17:15:05.000000,93483
3,1542820616,tagir_analyzes,10564,large,341,2025-09-04 18:22:28.000000,23abbd2e-5416-552c-aedb-738430bf0c48,user,user,user,0,2025-09-03 17:15:05.000000,90443
4,1542820616,tagir_analyzes,10564,large,341,2025-09-04 09:30:32.000000,408e9ba0-56a4-5fbe-bfd7-93e6edb952c5,tagir_analyzes,Тагир Анализирует,channel,1,2025-09-03 17:15:05.000000,58527
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71573,2351303932,Ai_bolno_ml,675,lower_medium,38,2025-06-29 14:59:53.000000,4f970b83-4471-59c3-8f4a-a1508ab71c51,user,user,user,0,2025-06-29 13:21:01.000000,5932
71574,2351303932,Ai_bolno_ml,675,lower_medium,40,2025-07-02 16:27:52.000000,c2fbf2ba-0097-5357-b84a-e60b4674bebd,n_it_girls,N айтишниц заходят в бар,channel,0,2025-07-02 08:39:17.000000,28115
71575,2351303932,Ai_bolno_ml,675,lower_medium,40,2025-07-02 10:43:48.000000,56ccd771-b59a-583d-bd84-41e49666168f,user,user,user,0,2025-07-02 08:39:17.000000,7471
71576,1996608001,Vkatysh_s_nulya,1266,lower_medium,117,2025-09-06 09:18:32.000000,a87e61d7-8bf2-5423-b435-f0aaac70dde6,dima_sqlit,Дима SQL-ит 🧑‍💻 (Аналитика данных),channel,0,2025-09-05 07:04:02.000000,94470


In [3]:
df_comments["comment_date"] = pd.to_datetime(df_comments["comment_date"])
df_comments.sort_values(by="comment_date", inplace=True)

In [4]:
df_external = df_comments.query("is_internal == 0").copy()
df_external.reset_index(drop=True, inplace=True)

(df_external
 .groupby("author_type")
    .agg(count=("author_uuid", "count"))
    .assign(percent=lambda x: (x["count"] / x["count"].sum() * 100).round(2))
 )

,count,percent
author_type,,
channel,17281,29.22
user,41861,70.78


In [5]:
df_external_agg = (df_external
                   .query("author_type == 'channel'")
                   .groupby(["author_type", "author_title"])
                    .agg(
                        comments_count=("author_type", "count"),
                        time_to_comment_mean=("time_to_comment_sec","mean"),
                        lover_small=("channel_id", lambda x: x[df_external.loc[x.index, "channel_segment"]=="lover_small"].nunique()),
                        upper_small=("channel_id", lambda x: x[df_external.loc[x.index, "channel_segment"]=="upper_small"].nunique()),
                        lower_medium=("channel_id", lambda x: x[df_external.loc[x.index, "channel_segment"]=="lower_medium"].nunique()),
                        upper_medium=("channel_id", lambda x: x[df_external.loc[x.index, "channel_segment"]=="upper_medium"].nunique()),
                        large=("channel_id", lambda x: x[df_external.loc[x.index, "channel_segment"]=="large"].nunique()),
                        top_segment_by_comments=("channel_segment", lambda x: x.value_counts().idxmax())
                    )
                    .assign(
                        comments_percent=lambda x: (x["comments_count"] / x["comments_count"].sum() * 100).round(2),
                        top_segment_by_comments_cnt=lambda x: x["top_segment_by_comments"].map({
                            "lover_small": "1–100",
                            "upper_small": "100–500",
                            "lower_medium": "500–2000",
                            "upper_medium": "2000–10000",
                            "large": "10000+"}),
                        color_label=lambda x: pd.cut(
                            x["comments_percent"],
                            bins=[-1, 1, 5, 10, 100],   # интервалы: 0–1%, 1–5%, 5–10%, >10%
                            labels=["<1%", "1–5%", "5–10%", "10%+"])
                    )
                   .reset_index()
                   )

df_external_agg

,author_type,author_title,comments_count,time_to_comment_mean,lover_small,upper_small,lower_medium,upper_medium,large,top_segment_by_comments,comments_percent,top_segment_by_comments_cnt,color_label
0,channel,(YaA) Yet another Analytic,20,3192.450000,0,0,1,0,0,lower_medium,0.12,500–2000,<1%
1,channel,... И ещё я побегал | Павел Потапов,15,16783.266667,0,0,1,1,0,lower_medium,0.09,500–2000,<1%
2,channel,.......,1,1212.000000,0,0,0,0,1,large,0.01,10000+,<1%
3,channel,1,26,40205.076923,0,0,3,1,0,lower_medium,0.15,500–2000,<1%
4,channel,23,5,460735.000000,0,0,0,1,1,large,0.03,10000+,<1%
...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,channel,👨🏻‍💻Кобозев Дмитрий | Создание сайтов,1,2410.000000,0,0,0,1,0,upper_medium,0.01,2000–10000,<1%
653,channel,💚 Just Hope | О шитье и айти 💚,2,87568.000000,0,0,1,0,1,lower_medium,0.01,500–2000,<1%
654,channel,💥Злата☀️,1,9802.000000,0,0,0,0,1,large,0.01,10000+,<1%
655,channel,💸Big Money Mommy💸,1,1097.000000,0,0,0,1,0,upper_medium,0.01,2000–10000,<1%


In [6]:
df_external_agg["label"] = (
        df_external_agg["author_title"] + "<br>"
        + "- комментов: " + df_external_agg["comments_count"].astype(str) + " (" + df_external_agg["comments_percent"].astype(str)  + "%)" + "<br>"
        + "- любимые каналы: " + df_external_agg["top_segment_by_comments_cnt"].astype(str)
)

In [47]:
def show_treemap_with_legend(data, path):
    fig = make_subplots(
        rows=2, cols=1,
        row_heights=[0.05, 0.95],
        vertical_spacing=0.0,
        specs=[[{"type": "xy"}], [{"type": "treemap"}]]
    )

    # Цвета и подписи для легенды
    legend_colors = {
        "10%+": "#93360e",
        "5–10%": "#5c5c34",
        "1–5%": "#97864c",
        "<1%": "#be8830"
    }

    legend_labels = list(legend_colors.keys())
    n = len(legend_labels)

    for i, (label, color) in enumerate(legend_colors.items()):
        # прямоугольник
        fig.add_trace(go.Scatter(
            x=[i, i + 1, i + 1, i, i],
            y=[0, 0, 1, 1, 0],
            fill="toself",
            fillcolor=color,
            line=dict(color=color),
            mode='lines',
            showlegend=False
        ), row=1, col=1)

        # текст поверх прямоугольника
        fig.add_trace(go.Scatter(
            x=[i + 0.5],
            y=[0.5],
            mode='text',
            text=f"Доля комментариев {label}",
            textfont=dict(size=14, color="white"),
            showlegend=False
        ), row=1, col=1)

    # основной treemap
    treemap_fig = px.treemap(
        data,
        path=path,
        values="comments_count",
        color="color_label",
        color_discrete_map=legend_colors
    )

    for trace in treemap_fig.data:
        fig.add_trace(trace, row=2, col=1)

    fig.update_layout(
        width=1080,
        height=1080,
        margin=dict(t=20, b=20, l=20, r=20),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=[0, n]),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=[0, 1]),
        plot_bgcolor='white',
        paper_bgcolor='white'
    )

    fig.update_traces(textfont=dict(size=16, color="white"), selector=dict(type='treemap'))
    fig.show()

    pio.write_image(
        fig,
        "treemap.png",
        scale=10,
        width=1080, height=1080
    )

show_treemap_with_legend(data=df_external_agg, path=["label"])

## Каналы, баовые агрегаты

In [148]:
%%sql
-- Агергация всех данных по каналу-посту
with
    comments_prep as (
        select
            com.channel_id,
            com.post_id,
            count(*) as comments,
            min(com.comment_date) as comments_ts_min,
            max(com.comment_date) as comments_ts_max,
            strftime('%s', max(com.comment_date)) - strftime('%s', min(com.comment_date)) as comments_ts_diff_sec,

            -- Все комментарии
            count(*) filter (where com.author_username='user') as comments_user,
            min(com.comment_date) filter (where com.author_username='user') as comments_user_ts_min,
            max(com.comment_date) filter (where com.author_username='user') as comments_user_ts_max,
            (
                strftime('%s', max(com.comment_date) filter (where com.author_username='user'))
                -
                strftime('%s', min(com.comment_date) filter (where com.author_username='user'))
            ) as comments_user_ts_diff_sec,

            -- Комментарии пользователей
            count(*) filter (where com.author_username!='user') as comments_channel,
            min(com.comment_date) filter (where com.author_username!='user') as comments_channel_ts_min,
            max(com.comment_date) filter (where com.author_username!='user') as comments_channel_ts_max,
            (
                strftime('%s', max(com.comment_date) filter (where com.author_username!='user'))
                -
                strftime('%s', min(com.comment_date) filter (where com.author_username!='user'))
            ) as comments_channel_ts_diff_sec,

            -- Комментарии канала автора
            count(*) filter (where com.author_username!='user' and com.author_username==cha.channel_name) as comments_channel_internal,
            min(com.comment_date) filter (where com.author_username!='user' and com.author_username==cha.channel_name) as comments_channel_internal_ts_min,
            max(com.comment_date) filter (where com.author_username!='user' and com.author_username==cha.channel_name) as comments_channel_internal_ts_max,
            (
                strftime('%s', max(com.comment_date) filter (where com.author_username!='user' and com.author_username==cha.channel_name))
                -
                strftime('%s', min(com.comment_date) filter (where com.author_username!='user' and com.author_username==cha.channel_name))
            ) as comments_channel_internal_ts_diff_sec,

            -- Комментарии канала не автора
            count(*) filter (where com.author_username!='user' and com.author_username!=cha.channel_name) as comments_channel_external,
            min(com.comment_date) filter (where com.author_username!='user' and com.author_username!=cha.channel_name) as comments_channel_external_ts_min,
            max(com.comment_date) filter (where com.author_username!='user' and com.author_username!=cha.channel_name) as comments_channel_external_ts_max,
            (
                strftime('%s', max(com.comment_date) filter (where com.author_username!='user' and com.author_username!=cha.channel_name))
                -
                strftime('%s', min(com.comment_date) filter (where com.author_username!='user' and com.author_username!=cha.channel_name))
            ) as comments_channel_external_ts_diff_sec

        from comments as com
        left join channels as cha on com.channel_id=cha.channel_id
        group by com.channel_id, com.post_id
    ),

    posts_prep as (
        select
            p.*,
            cp.*,
            strftime('%s', cp.comments_ts_min) - strftime('%s', p.post_date)  as post2comments_ts_diff_sec

        from posts as p
        left join comments_prep as cp on p.channel_id=cp.channel_id and p.post_id=cp.post_id
    ),

    channels_prep as (
        select
            channel_id,
            channel_name,
            subscribers as channel_subscribers,
            case
                when subscribers < 100 then '1_lover_small'
                when subscribers < 500 then '2_upper_small'
                when subscribers < 2000 then '3_lower_medium'
                when subscribers < 10000 then '4_upper_medium'
                else '5_large'
            end as channel_segment
        from channels
        where is_target=1
    ),

    result as (
        select
            pp.channel_id,
            pp.channel_name,
            pp.channel_title,
            cp.channel_subscribers,
            cp.channel_segment,
            pp.post_id,
            pp.post_preview,
            pp.post_views,
            pp.post_forwards,
            pp.post_replies,
            pp.post_date as post_ts,
            pp.post2comments_ts_diff_sec,
            pp.comments,
            pp.comments_ts_min,
            pp.comments_ts_max,
            pp.comments_ts_diff_sec,
            pp.comments_user,
            pp.comments_user_ts_min,
            pp.comments_user_ts_max,
            pp.comments_user_ts_diff_sec,
            pp.comments_channel,
            pp.comments_channel_ts_min,
            pp.comments_channel_ts_max,
            pp.comments_channel_ts_diff_sec,
            pp.comments_channel_internal,
            pp.comments_channel_internal_ts_min,
            pp.comments_channel_internal_ts_max,
            pp.comments_channel_internal_ts_diff_sec,
            pp.comments_channel_external,
            pp.comments_channel_external_ts_min,
            pp.comments_channel_external_ts_max,
            pp.comments_channel_external_ts_diff_sec

        from posts_prep as pp
        inner join channels_prep as cp on pp.channel_id=cp.channel_id
    )

select * from result;

,channel_id,channel_name,channel_title,channel_subscribers,channel_segment,post_id,post_preview,post_views,post_forwards,post_replies,...,comments_channel_ts_max,comments_channel_ts_diff_sec,comments_channel_internal,comments_channel_internal_ts_min,comments_channel_internal_ts_max,comments_channel_internal_ts_diff_sec,comments_channel_external,comments_channel_external_ts_min,comments_channel_external_ts_max,comments_channel_external_ts_diff_sec
0,1178238337,boris_again,Борис опять,15111,5_large,3327,Адам Кадыров получил новую награду: test of ti...,13514,102,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1178238337,boris_again,Борис опять,15111,5_large,3328,Papers with code закрыли :(\n\nhttps://papersw...,5363,72,5,...,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,1178238337,boris_again,Борис опять,15111,5_large,3329,"🚀 Уважаемые коллеги, кому интересна математик...",4767,59,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1178238337,boris_again,Борис опять,15111,5_large,3330,**Gemini 2.5 Flash Lite\n\n**Пару дней назад [...,5088,39,1,...,2025-07-26 11:36:09.000000,0.0,0.0,NaN,NaN,NaN,1.0,2025-07-26 11:36:09.000000,2025-07-26 11:36:09.000000,0.0
4,1178238337,boris_again,Борис опять,15111,5_large,3331,А вы тоже видите в этом здании младшего брата ...,5897,27,8,...,2025-07-26 11:47:27.000000,203.0,2.0,2025-07-26 11:44:04.000000,2025-07-26 11:47:27.000000,203.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5495,2864348643,davidzagorski,"Загорски, работай! [Данные и Проекты]🚀",16,1_lover_small,19,Надо и мемов добавлять в разнообразие.\nПомню ...,96,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5496,2864348643,davidzagorski,"Загорски, работай! [Данные и Проекты]🚀",16,1_lover_small,20,"В прекрасном Karpov.Courses поделились постом,...",63,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5497,2864348643,davidzagorski,"Загорски, работай! [Данные и Проекты]🚀",16,1_lover_small,21,"Стараюсь влево и вправо не уходить, но иногда ...",44,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5498,2864348643,davidzagorski,"Загорски, работай! [Данные и Проекты]🚀",16,1_lover_small,22,"Решил, что хочется подтянуть свой уровень влад...",48,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
